In [ ]:
# AO* Algorithm

def get_neighbors(graph, node):
    return graph.get(node, [])

def compute_min_cost_child_nodes(graph, heuristic, node):
    neighbors = get_neighbors(graph, node)
    if not neighbors:  # Goal node
        return heuristic[node], []

    min_cost = float('inf')
    best_child_group = None

    for child_group in neighbors:   # Each group = [(child, cost), ...]
        cost = 0
        for child, edge_cost in child_group:
            cost += edge_cost + heuristic[child]   # f = g + h
        if cost < min_cost:
            min_cost = cost
            best_child_group = child_group

    return min_cost, best_child_group

def ao_star(graph, heuristic, start, goals):
    status = {}                                  #Tracks whether a node is solved ("Solved") or not yet.
    solution = {}                                #For each node, stores the chosen best child-group (list of children) giving the currently best solution.
    parent = {}                                  #can backtrack and update parents when a child’s cost changes / becomes solved.

    def recursive_ao(node, backtracking=False):
        print(f"Processing node: {node}")

        if node in goals:  # Goal node
            status[node] = "Solved"
            return

        cost, child_group = compute_min_cost_child_nodes(graph, heuristic, node)
        heuristic[node] = cost
        solution[node] = [child for child, _ in child_group]

        for child, _ in child_group:
            parent[child] = node

        solved = True
        for child, _ in child_group:
            if child not in status or status[child] != "Solved":
                solved = False

        if solved:
            status[node] = "Solved"

        if not backtracking:  # Expand children -forward move
            for child, _ in child_group:
                recursive_ao(child, backtracking=True)

        if parent.get(node):  # Backtrack
            recursive_ao(parent[node], backtracking=True)

    recursive_ao(start)
    return solution


# ---------------- Example ----------------
graph = {
    'A': [[('B', 1), ('C', 1)], [('D', 1)]],   # A → (B,C) OR D
    'B': [[('E', 1)], [('F', 1)]],             # B → E OR F
    'C': [[('G', 1)]],                         # C → G
    'D': [[('H', 1)]]                          # D → H
}

heuristic = {
    'A': 10, 'B': 6, 'C': 4, 'D': 2,
    'E': 0, 'F': 0, 'G': 0, 'H': 0   # Goal nodes
}

goals = {'E', 'F', 'G', 'H'}

solution = ao_star(graph, heuristic, 'A', goals)

print("\nOptimal Solution Graph:")
for node in solution:
    print(f"{node} -> {solution[node]}")


Processing node: A
Processing node: D
Processing node: A

Optimal Solution Graph:
A -> ['D']
D -> ['H']
